In [1]:
import pandas as pd

In [2]:
gtf_file = "/mnt/xomics/tabear/atlas_paper/Neural-Retina-Atlas/data/ref_genome/gencode.v39.primary_assembly.annotation.gtf"
gtf_columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
gtf_df = pd.read_csv(gtf_file, sep='\t', comment='#', header=None, names=gtf_columns)


In [12]:
gtf_df['attribute'][0]

'gene_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; level 2; hgnc_id "HGNC:37102"; havana_gene "OTTHUMG00000000961.2";'

In [22]:
# Filter for exons only
exons_df = gtf_df[gtf_df['feature'] == 'exon']

# Extract gene_id from the attribute column
exons_df['gene_id'] = exons_df['attribute'].str.extract('gene_id "([^"]+)"')

# Count the number of exons per gene
exon_counts = exons_df.groupby('gene_id').size()

# Filter for genes with only one exon
single_exon_genes = exon_counts[exon_counts == 1].index

#Filter the original GTF for these genes
filtered_gtf_df = gtf_df[gtf_df['attribute'].str.contains('|'.join(single_exon_genes))]

/tmp/ipykernel_361148/2037249361.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exons_df['gene_id'] = exons_df['attribute'].str.extract('gene_id "([^"]+)"')


In [33]:
genes = filtered_gtf_df[filtered_gtf_df['feature'] == 'gene']
# Extract gene_id and remove version numbers (if present)
genes['gene'] = genes['attribute'].str.extract('gene_id "([^"]+)"')[0].apply(lambda x: x.split('.')[0])
genes.head()

/tmp/ipykernel_361148/403311177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genes['gene'] = genes['attribute'].str.extract('gene_id "([^"]+)"')[0].apply(lambda x: x.split('.')[0])


,seqname,source,feature,start,end,score,strand,frame,attribute,gene
25,chr1,ENSEMBL,gene,17369,17436,.,-,.,"gene_id ""ENSG00000278267.1""; gene_type ""miRNA""...",ENSG00000278267
36,chr1,ENSEMBL,gene,30366,30503,.,+,.,"gene_id ""ENSG00000284332.1""; gene_type ""miRNA""...",ENSG00000284332
47,chr1,HAVANA,gene,52473,53312,.,+,.,"gene_id ""ENSG00000268020.3""; gene_type ""unproc...",ENSG00000268020
96,chr1,HAVANA,gene,131025,134836,.,+,.,"gene_id ""ENSG00000233750.3""; gene_type ""proces...",ENSG00000233750
99,chr1,HAVANA,gene,135141,135895,.,-,.,"gene_id ""ENSG00000268903.1""; gene_type ""proces...",ENSG00000268903


In [28]:
# Save the filtered GTF
genes.to_csv("single_exon_genes.gtf", sep='\t', index=False, header=False)

In [34]:
isoquant = pd.read_csv('/mnt/xomics/tabear/atlas_paper/Neural-Retina-Atlas/data/transcriptomics/isoquant/isoquant_classification.tsv', sep = '\t')
isoquant.head()

,Unnamed: 0,isoform,length,structural_category,associated_gene,associated_transcript,subcategory,FL_count,gene_symbol
0,0,TRANSCRIPT39.CHR1.NNIC,3095,novel_not_in_catalog,ENSG00000227232,TRANSCRIPT39.CHR1.NNIC,"intron_alternation_novel:14830-14969,exon_gain...",8.0,WASH7P
1,1,TRANSCRIPT89.CHR1.NNIC,7549,novel_not_in_catalog,ENSG00000227232,TRANSCRIPT89.CHR1.NNIC,"intron_alternation_novel:14830-14969,alt_donor...",22.0,WASH7P
2,2,TRANSCRIPT91.CHR1.NNIC,1796,novel_not_in_catalog,ENSG00000227232,TRANSCRIPT91.CHR1.NNIC,"intron_alternation_novel:14830-14969,terminal_...",21.0,WASH7P
3,3,ENST00000469289.1,535,full-splice_match,ENSG00000243485,ENST00000469289.1,NaN,1.0,MIR1302-2HG
4,4,ENST00000417324.1,1187,full-splice_match,ENSG00000237613,ENST00000417324.1,NaN,125.0,FAM138A


In [37]:
monoexonic = isoquant[isoquant['associated_gene'].isin(genes['gene'])]
monoexonic.to_csv('monoexonic_isoquant.tsv', sep = '\t', index = False)